In [9]:
import speech_recognition as sr
import pyttsx3
import string
import pickle
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from util import JSONParser

# Fungsi untuk mengenali suara dan mengubahnya menjadi teks
def recognize_speech_from_mic(recognizer, microphone):
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be an instance of `Recognizer`")
    if not isinstance(microphone, sr.Microphone):
        raise TypeError("`microphone` must be an instance of `Microphone`")

    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        # print("Saya mendengarkankan anda......")
        print(" ")
        try:
            audio = recognizer.listen(source)
        except sr.WaitTimeoutError:
            response = {
                "success": False,
                "error": None
            }
            return response

    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    try:
        response["transcription"] = recognizer.recognize_google(audio, language='id-ID')
    except sr.RequestError:
        response["success"] = False
        response["error"] = "API unavailable"
    except sr.UnknownValueError:
        response["error"] = "Bisakah anda berbicara lagi, tuan!"

    return response

# Fungsi untuk praproses teks
def preprocess(chat):
    chat = chat.lower()
    tandabaca = tuple(string.punctuation)
    chat = ''.join(ch for ch in chat if ch not in tandabaca)
    return chat

# Fungsi untuk mendapatkan respons dari chatbot
def bot_response(chat, pipeline, jp):
    chat = preprocess(chat)
    res = pipeline.predict_proba([chat])
    max_prob = max(res[0])
    if max_prob < 0.2:
        return "Maaf, tuan. Saya tidak dapat menjawab! :(", None
    else:
        max_id = np.argmax(res[0])
        pred_tag = pipeline.classes_[max_id]
        return jp.get_response(pred_tag), pred_tag

# Fungsi untuk mengubah teks menjadi suara
def text_to_speech(teks):
    engine = pyttsx3.init()
    engine.say(teks)
    engine.runAndWait()

# Load model chatbot
with open("model_chatbot.pkl", "rb") as model_file:
    pipeline = pickle.load(model_file)

# Load data intents
path = "data/intents.json"
jp = JSONParser()
jp.parse(path)

def main():
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    print("Mikrofon siap, tekan Ctrl+C untuk berhenti.")
    while True:
        try:
            response = recognize_speech_from_mic(recognizer, microphone)
            if response["transcription"]:
                print("Anda  : " + response["transcription"])
                res, tag = bot_response(response["transcription"], pipeline, jp)
                print(f"Jarvis: {res}")
                text_to_speech(res)
                if tag == 'bye':
                    break
            if not response["success"]:
                print("Gagal mengenali suara")
            if response["error"]:
                print(response["error"])
        except KeyboardInterrupt:
            print("\nProgram dihentikan.")
            break

if __name__ == "__main__":
    main()

[INFO] Data JSON converted to DataFrame with shape : (44, 2)
Mikrofon siap, tekan Ctrl+C untuk berhenti.


Anda  : halo
Jarvis: Halo!


Anda  : Siapa nama kamu
Jarvis: Halo, aku JARVIS!


Anda  : Untuk apa kamu dibuat
Jarvis: Aku membantu anda, Tuan. Untuk melakukan pekerjaan sehari-hari


Anda  : apa yang bisa kamu lakukan
Jarvis: Saya bisa membuat, menghapus, menyalin, membuka, dan menutup file serta aplikasi. Untuk anda!


Anda  : Oke sampai di sini saja
Jarvis: Jika anda butuh bantuan lagi, saya siap membantu!
